## Spark SQL

### Imports

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *

### Spark Session erstellen

In [3]:
spark = SparkSession \
        .builder \
        .appName("Python Spark SQL example") \
        .getOrCreate()

### Daten einlesen

In [4]:
df = spark.read.json("people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Ausgabe des Schemas

In [5]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### Übung 1

Selektiere nur die Spalte "Name"

In [6]:
df.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



### Übung 2

Selektiere alles und erhöhe das Alter um 1

In [7]:
df.select(df['name'], df['age'] + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



### Übung 3

Zähle die Personen nach alter

In [15]:
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



### Übung 4

Registrieren des DataFrame als SQL temporary view

In [10]:
df.createOrReplaceTempView("people")
sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Übung 5

Registrieren des DataFrame als global temporary view

In [11]:
df.createGlobalTempView("people")

### Übung 6

Abfrage der global temporary view, diese view hat den prefix "global_temp"

In [12]:
spark.sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Übung 7

Global temporary view sind cross-session. Erzeuge eine neue Session und gibt die view aus

In [13]:
spark.newSession().sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Übung 8

Lade das people.txt file

In [16]:
sc = spark.sparkContext
lines = sc.textFile("people.txt")

### Übung 9

Konvertiere pro Linie den Namen und das Alter in eine Tuple

In [17]:
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: (p[0], p[1].strip()))

### Übung 10

Erstelle ein Schema um die Personendaten in ein Dataframe umzuwandeln

In [18]:
schemaString = "name age"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

### Übung 11

Erstelle ein Datframe und wende das Schema an und gibt die Daten des Dataframe aus

In [20]:
schemaPeople = spark.createDataFrame(people, schema)
schemaPeople.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+

